# Steps
1. Choose data scenario.
2. Generate the dataset accordingly.
5. Choose 'fixed' configuration.
6. For each 'fixed' configuration, optimize the other parameters based on RMSE.
7. Given optimal setting, run popularity bias analysis for every version of the 'fixed' configuration.

## Libraries

In [ ]:
%env MKL_THREADING_LAYER=tbb
%env OPENBLAS_NUM_THREADS=24
%env NUMBA_NUM_THREADS=96
%env MKL_NUM_THREADS=96
%env OMP_NUM_THREADS=1

In [ ]:
import os
os.environ["MKL_THREADING_LAYER"] = "tbb"
os.environ["OPENBLAS_NUM_THREADS"] = '24'
os.environ["NUMBA_NUM_THREADS"] = '96'
os.environ["MKL_NUM_THREADS"] = '96'
os.environ["OMP_NUM_THREADS"] = '1'
# for random generation
import numpy as np 
import random as rd


# basic functions
import pandas as pd
pd.options.display.max_rows = 100
import pickle

# custom-made functions
import modelling_mf
from data_generation import generate_data
from optimize_hp import optimize_lkpy, optimize_cornac

# lenskit RS library
from lenskit.algorithms import user_knn


# cornac RS library
from cornac.models import UserKNN

## Fairbook data and parameters

In [ ]:
data="fairbook"
# user-item interactions
fairbook_ratings = pd.read_csv("data/"+data+"_events.csv")

In [ ]:
user_col = "user" # the name of the column that includes the users
item_col = "item" # the name of the column that includes the items
predict_col="rating" # the name of the column that includes the interaction

## Scenarios

1. There is no general relation between popularity and rating: uniformly_random
2. Popular items are rated higher by the users.: popularity_good
3. Popular items are rated lower by the users.: popilarity_bad
4. Popular items are rated higher by users with big profiles.: popularity_good_for_bp_ur
5. Popular items are rated lower by users with big profiles.: popularity_bad_for_bp_ur

In [ ]:
data_strategies = ['uniformly_random',
                   'popularity_good',
                   'popularity_bad',
                   'popularity_good_for_bp_ur',
                   'popularity_bad_for_bp_ur']

## Optimize, train, evaluate LKPY
- **Algorithm**
- **Fixed parameters**
- **To-optimize parameters**

In [ ]:
algo_versions = {"UserKNN":[{'min_nbrs':1, 'min_sim':0},
                            {'min_nbrs':2, 'min_sim':0},
                            {'min_nbrs':10, 'min_sim':0},
                            {'min_nbrs':1, 'min_sim':-1},
                            {'min_nbrs':2, 'min_sim':-1},
                            {'min_nbrs':10, 'min_sim':-1}]}

In [ ]:
evaluation_way = "cross_validation"
verbose = False
plot = True
save_plot = True # save the plots
fallback = False
nr_recs = 10
sampling_strategy = "frac"
partition_way = "user"

In [ ]:
# choose algorithm
algorithm_lkpy = user_knn.UserUser
algo_name = "UserKNN"
versions = algo_versions[algo_name]

# for every data strategy
for i in range(len(data_strategies)):
    data_strategy = data_strategies[i]
    # generate the data 
    ratings = generate_data(strategy = data_strategy,
                            copying_dataset = fairbook_ratings,
                            user_perc = 0.2)
    
    # for every 'fixed' version of the algorithm
    for args in versions:
        print(data_strategy, args)
        
        # optimize for this fixed version
        best_params = optimize_lkpy(ratings=ratings, algorithm_name=algo_name, args=args, partition_way='row', max_evals=20)

        # save the best parameters for this fixed version

        with open('best_parameters/'+algo_name+'/'+data_strategy+'_'+str(args)+'.pkl', 'wb') as f:
            pickle.dump(best_params, f)


        optimal_nnbrs = best_params["nnbrs"]

        

        # run the training and evaluation for the fixed version + the best other parameters
        pop_biases_lkpy, metrics_dict_lkpy = modelling_mf.train_algorithm(algorithm = algorithm_lkpy(nnbrs=optimal_nnbrs,
                                                                
                                                                center=True,
                                                            min_sim=args['min_sim'],
                                                            min_nbrs=args['min_nbrs']),
                                                        algo_name = algo_name,  
                                                        ratings = ratings,
                                                        evaluation_way = evaluation_way,
                                                        verbose = verbose, 
                                                        n=nr_recs,
                                                        sampling_strategy = sampling_strategy,
                                                        partition_way = partition_way,
                                                        plot = plot,
                                                    data_strategy=data_strategy,
                                                    args=args,
                                                    save_plot=save_plot)

        # Save metrics!
        with open('experimental_results/'+algo_name+'/'+data_strategy+'_'+str(args)+'.pkl', 'wb') as f:
            pickle.dump(metrics_dict_lkpy, f)

## Optimize, train, evaluate Cornac
- **Algorithm**
- **Fixed parameters**
- **To-optimize parameters**

In [ ]:
algo_versions = {"CornacUserKNN":[{'center':True}],
                }

In [ ]:
algorithm_cornac = UserKNN
algo_name = "CornacUserKNN"
versions = algo_versions[algo_name]

# for every data strategy
for i in range(len(data_strategies)):
    data_strategy = data_strategies[i]
    # generate the data 
    ratings = generate_data(strategy = data_strategy,
                            copying_dataset = fairbook_ratings,
                            user_perc = 0.2)
    # for every 'fixed' version of the algorithm
    for args in versions:
        print(data_strategy, args)
        
        # optimize for this fixed version
        best_params = optimize_cornac(ratings=ratings, algorithm_name=algo_name, args=args, max_evals=20)

        # save the best parameters for this fixed version

        with open('best_parameters/'+algo_name+'/'+data_strategy+'_'+str(args)+'.pkl', 'wb') as f:
            pickle.dump(best_params, f)


        optimal_k = best_params['k']

        

        pop_biases_cornac, metrics_dict_cornac = modelling_mf.train_algorithm_cornac(algorithm = algorithm_cornac(k=optimal_k,
                                                                                                                 mean_centered=args['center']),
                                                        algo_name = algo_name,  
                                                        ratings = ratings,
                                                        evaluation_way = evaluation_way,
                                                        verbose = verbose, 
                                                        n=nr_recs,
                                                        sampling_strategy = sampling_strategy,
                                                        partition_way = partition_way,
                                                        plot = plot,
                                                    data_strategy=data_strategy,
                                                    args=args,
                                                    save_plot=save_plot)

        # Save metrics!
        with open('experimental_results/'+algo_name+'/'+data_strategy+'_'+str(args)+'.pkl', 'wb') as f:
            pickle.dump(metrics_dict_cornac, f)